In [12]:
import os

# Specify the directory where your images are stored
dataset_dir = r'C:\Users\Dell\OneDrive\Desktop\database\Plant_disease_detection\archive (6)\New Plant Diseases Dataset(Augmented)\New Plant Diseases Dataset(Augmented)'

# Supported image file extensions
valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif', '.ppm', '.pgm', '.webp')

# Function to check file extensions in each class folder
def check_file_extensions(directory):
    invalid_files = []
    
    # Walk through all subdirectories and files
    for root, dirs, files in os.walk(directory):
        for file in files:
            if not file.lower().endswith(valid_extensions):
                invalid_files.append(os.path.join(root, file))
    
    return invalid_files

# Check the train and valid datasets
invalid_files_train = check_file_extensions(os.path.join(dataset_dir, 'train'))
invalid_files_valid = check_file_extensions(os.path.join(dataset_dir, 'valid'))

# Print invalid files if found
if invalid_files_train:
    print("Invalid files in 'train' folder:")
    for f in invalid_files_train:
        print(f)

if invalid_files_valid:
    print("Invalid files in 'valid' folder:")
    for f in invalid_files_valid:
        print(f)

if not invalid_files_train and not invalid_files_valid:
    print("All files have valid image extensions.")


All files have valid image extensions.


In [13]:
import os

def check_empty_folders(directory):
    empty_folders = []
    
    # Walk through all subdirectories
    for root, dirs, files in os.walk(directory):
        if len(files) == 0 and len(dirs) == 0:  # No files and no subfolders
            empty_folders.append(root)
    
    return empty_folders

# Check for empty folders in 'train' and 'valid' directories
empty_folders_train = check_empty_folders(os.path.join(dataset_dir, 'train'))
empty_folders_valid = check_empty_folders(os.path.join(dataset_dir, 'valid'))

# Print empty folders if found
if empty_folders_train:
    print("Empty folders in 'train' folder:")
    for folder in empty_folders_train:
        print(folder)

if empty_folders_valid:
    print("Empty folders in 'valid' folder:")
    for folder in empty_folders_valid:
        print(folder)

if not empty_folders_train and not empty_folders_valid:
    print("No empty folders found.")


No empty folders found.


In [1]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os

# Image Transformations (Normalization for RGB Images)
image_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 for better model compatibility
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Standard normalization for RGB
])

# Load Plant Disease Datasets
print(os.listdir(r'C:\Users\Dell\OneDrive\Desktop\database\Plant_disease_detection\archive (6)\New Plant Diseases Dataset(Augmented)\New Plant Diseases Dataset(Augmented)\train'))
train_dataset = datasets.ImageFolder(root=r'C:\Users\Dell\OneDrive\Desktop\database\Plant_disease_detection\archive (6)\New Plant Diseases Dataset(Augmented)\New Plant Diseases Dataset(Augmented)\train', transform=image_transforms)
test_dataset = datasets.ImageFolder(root=r'C:\Users\Dell\OneDrive\Desktop\database\Plant_disease_detection\archive (6)\New Plant Diseases Dataset(Augmented)\New Plant Diseases Dataset(Augmented)\valid', transform=image_transforms)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=True)

# Define CNN Model for Plant Disease Detection
class CNN_plant_disease(nn.Module):
    def __init__(self):
        super(CNN_plant_disease, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3)  # 222x222
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)  # 111x111
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3)  # 109x109
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)  # 54x54
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)  # 52x52
        self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)  # 26x26
        self.fc1 = nn.Linear(64 * 26 * 26, 512)  # Fully connected layer
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, len(train_dataset.classes))  # Output layer for plant disease classes
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)  # Prevent overfitting

    def forward(self, X):
        X = self.relu(self.maxpool1(self.conv1(X)))
        X = self.relu(self.maxpool2(self.conv2(X)))
        X = self.relu(self.maxpool3(self.conv3(X)))
        X = X.view(X.size(0), -1)  # Flatten
        X = self.relu(self.fc1(X))
        X = self.dropout(self.relu(self.fc2(X)))
        X = self.fc3(X)
        return X

# Initialize Model, Loss Function, and Optimizer
model = CNN_plant_disease()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
num_epochs = 10
for epoch in range(1, num_epochs + 1):
    model.train()
    total_loss = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()  # Reset gradients
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f"Epoch {epoch}/{num_epochs}, Loss: {total_loss:.4f}")

# Evaluation Function
def evaluate(loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in loader:
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='micro')
    recall = recall_score(all_labels, all_preds, average='micro')
    f1 = f1_score(all_labels, all_preds, average='micro')
    
    return accuracy, precision, recall, f1

# Evaluate Training and Testing Metrics
train_acc, train_prec, train_recall, train_f1 = evaluate(train_loader)
print("***Training Metrics***")
print(f"Accuracy: {train_acc:.4f}, Precision: {train_prec:.4f}, Recall: {train_recall:.4f}, F1-Score: {train_f1:.4f}")

test_acc, test_prec, test_recall, test_f1 = evaluate(test_loader)
print("***Testing Metrics***")
print(f"Accuracy: {test_acc:.4f}, Precision: {test_prec:.4f}, Recall: {test_recall:.4f}, F1-Score: {test_f1:.4f}")




['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___healthy', 'Corn_(maize)___Northern_Leaf_Blight', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___healthy', 'Potato___Late_blight', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___healthy', 'Strawberry___Leaf_scorch', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___healthy', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Target_Spot', 'Tomato___Tomato_mosaic_virus', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus']
Epoch 1/10, Loss: 1050.7787
Epoch 2/10, Loss: 288.

In [2]:
torch.save(model.state_dict(), 'plant_disease_detection_model.pth')
print("Model saved as 'emotion_detection_model.pth'")

Model saved as 'emotion_detection_model.pth'


In [3]:
len(train_dataset.classes)

35